# Coursera CapStone Project

In [43]:
import pandas as pd
import numpy as np
import json
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [2]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Part 1 : Scrapping the neighborhoods in Toronto from Wiki

In [28]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [29]:
df.describe()

,Postal Code,Borough,Neighborhood
count,180,180,103
unique,180,11,98
top,M2S,Not assigned,Downsview
freq,1,77,4


In [30]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

In [32]:
df.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M1W,North York,Downsview
freq,1,24,4


In [34]:
df.shape

(103, 3)

# Part 2 : Reading the Lat and Long from the CSV


In [35]:
df_coordinates = pd.read_csv('./Geospatial_Coordinates.csv')

In [36]:
df_coordinates.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [37]:
df_coordinates.shape 

(103, 3)

### Merging two dataframes on Postal Code column

In [38]:
df_neighbour_with_coordinates = df_coordinates.merge(df, on='Postal Code')

In [40]:
df_neighbour_with_coordinates.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [42]:
df_neighbour_with_coordinates.shape

(103, 5)

### Plot all neighbors on the map to visually inspect clustering

In [46]:
toronto_lat = 43.6532
toronto_long = -79.3832

In [50]:
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)


# add markers to map
for lat, lng, label in zip(df_neighbour_with_coordinates['Latitude'], df_neighbour_with_coordinates['Longitude'], df_neighbour_with_coordinates['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

### Observation - Downtown area is very dense 

In [51]:
map_toronto